In [1]:
import time
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
import torch
import transforms as T
from movinets.config import _C
import numpy as np
from movinets import MoViNet
import random
import gc

torch.manual_seed(97)
num_frames = 16 # 16
clip_steps = 2
Bs_Train = 16
Bs_Test = 16


In [ ]:
class_dict=dict()
class_dict['normal']=0
class_dict['assault']=1
class_dict['fight']=2
class_dict['burglary']=3
class_dict['vandalism']=4
class_dict['swoon']=5
class_dict['wander']=6
class_dict['trespass']=7
class_dict['dump']=8
class_dict['robbery']=9
class_dict['datefight']=10
class_dict['kidnap']=11
class_dict['drunken']=12

In [2]:
#i+1은 위 클래스 번호입니다.
data_arr=[]
label_arr=[]
for i in range(6):
    data_arr.append(np.load('../sampledata/sample_train_'+ str(i+1) +'_uint8.npy'))
    label_arr.append(np.load('../sampledata/sample_label_'+ str(i+1) +'_uint8.npy'))

In [3]:
for i in range(6):
    print(data_arr[i].shape)
    print(label_arr[i].shape)

(1523, 16, 172, 172, 3)
(1523,)
(1541, 16, 172, 172, 3)
(1541,)
(1526, 16, 172, 172, 3)
(1526,)
(1665, 16, 172, 172, 3)
(1665,)
(1545, 16, 172, 172, 3)
(1545,)
(1539, 16, 172, 172, 3)
(1539,)


In [4]:
data = np.concatenate((data_arr[0], data_arr[1], data_arr[2],
                       data_arr[3], data_arr[4], data_arr[5]))

In [5]:
del data_arr

In [6]:
label = np.concatenate((label_arr[0], label_arr[1], label_arr[2],
                        label_arr[3], label_arr[4], label_arr[5]))

In [7]:
audio=[]
for i in range(data.shape[0]):
    audio.append([])
    
audio = np.array(audio)

In [8]:
print(data.shape)
print(label.shape)
print(audio.shape)


(9339, 16, 172, 172, 3)
(9339,)
(9339, 0)


In [9]:
random_index=random.sample([i for i in range(data.shape[0])], int(data.shape[0]/10))

In [10]:
len(random_index)

933

In [11]:
val_data = torch.from_numpy(data[random_index].reshape(-1, 3, 16, 172, 172))
val_audio = torch.from_numpy(audio[random_index]).reshape(int(data.shape[0]/10),1,0)
val_label = torch.from_numpy(label[random_index])

In [12]:
train_data = torch.from_numpy(np.delete(data, random_index, axis=0).reshape(-1, 3, 16, 172, 172))
train_audio = torch.from_numpy(np.delete(audio, random_index, axis=0)).reshape(data.shape[0]-int(data.shape[0]/10),1,0)
train_label = torch.from_numpy(np.delete(label, random_index, axis=0))

In [13]:
print(train_data.shape)
print(train_audio.shape)
print(train_label.shape)

print(val_data.shape)
print(val_audio.shape)
print(val_label.shape)

torch.Size([8406, 3, 16, 172, 172])
torch.Size([8406, 1, 0])
torch.Size([8406])
torch.Size([933, 3, 16, 172, 172])
torch.Size([933, 1, 0])
torch.Size([933])


In [14]:
del data

In [15]:
#train = torch.from_numpy(train.reshape(-1, 3, 16, 172, 172))

In [16]:
#label = torch.from_numpy(label)

In [17]:
#train.shape

In [18]:
#train = torch.from_numpy(np.load('../sample_train_uint8.npy').reshape(9220, 3, 16, 172, 172))

In [19]:
#label = torch.from_numpy(np.load('../sample_label_uint8.npy'))

In [20]:
'''
audio=[]
for i in range(train.shape[0]):
    audio.append([])
'''

'\naudio=[]\nfor i in range(train.shape[0]):\n    audio.append([])\n'

In [21]:
'''
audio = torch.from_numpy(np.array(audio)).reshape(train.shape[0],1,0)
'''

'\naudio = torch.from_numpy(np.array(audio)).reshape(train.shape[0],1,0)\n'

In [22]:
class cctv():

    def __init__(self, train_data, audio_data, label_data):
        self.train = train_data
        self.audio = audio_data
        self.label = label_data

    def __len__(self):
        return self.train.shape[0]

    def __getitem__(self, idx):
        return self.train[idx].float()/255, self.audio[idx], self.label[idx].long()


In [23]:
cctv_train = cctv(train_data, train_audio, train_label)

In [24]:
cctv_val = cctv(val_data, val_audio, val_label)

In [25]:
train_loader = DataLoader(cctv_train, batch_size=Bs_Train, shuffle=True)

In [26]:
val_loader = DataLoader(cctv_val, batch_size=Bs_Train, shuffle=False)

In [27]:
def train_iter(model, optimz, data_load, loss_val):
    samples = len(data_load.dataset)
    model.train()
    model.cuda()
    model.clean_activation_buffers()
    optimz.zero_grad()
    for i, (data,_ , target) in enumerate(data_load):
        out = F.log_softmax(model(data.cuda()), dim=1)
        loss = F.nll_loss(out, target.cuda())
        loss.backward()
        optimz.step()
        optimz.zero_grad()
        model.clean_activation_buffers()
        if i % 50 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_load)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()))
            loss_val.append(loss.item())

def evaluate(model, data_load, loss_val):
    model.eval()
    
    samples = len(data_load.dataset)
    csamp = 0
    tloss = 0
    model.clean_activation_buffers()
    with torch.no_grad():
        for data, _, target in data_load:
            output = F.log_softmax(model(data.cuda()), dim=1)
            loss = F.nll_loss(output, target.cuda(), reduction='sum')
            _, pred = torch.max(output, dim=1)
            
            tloss += loss.item()
            csamp += pred.eq(target.cuda()).sum()
            model.clean_activation_buffers()
    aloss = tloss / samples
    loss_val.append(aloss)
    print('\nAverage loss: ' + '{:.4f}'.format(aloss) +
          '  Accuracy:' + '{:5}'.format(csamp) + '/' +
          '{:5}'.format(samples) + ' (' +
          '{:4.2f}'.format(100.0 * csamp / samples) + '%)\n')
    
def train_iter_stream(model, optimz, data_load, loss_val, n_clips = 2, n_clip_frames=8):
    """
    In causal mode with stream buffer a single video is fed to the network
    using subclips of lenght n_clip_frames. 
    n_clips*n_clip_frames should be equal to the total number of frames presents
    in the video.
    
    n_clips : number of clips that are used
    n_clip_frames : number of frame contained in each clip
    """
    #clean the buffer of activations
    samples = len(data_load.dataset)
    model.cuda()
    model.train()
    model.clean_activation_buffers()
    optimz.zero_grad()
    
    for i, (data,_, target) in enumerate(data_load):
        data = data.cuda()
        target = target.cuda()
        l_batch = 0
        #backward pass for each clip
        for j in range(n_clips):
            output = F.log_softmax(model(data[:,:,(n_clip_frames)*(j):(n_clip_frames)*(j+1)]), dim=1)
            loss = F.nll_loss(output, target)
            _, pred = torch.max(output, dim=1)
            loss = F.nll_loss(output, target)/n_clips
            loss.backward()
        l_batch += loss.item()*n_clips
        optimz.step()
        optimz.zero_grad()
        
        #clean the buffer of activations
        model.clean_activation_buffers()
        if i % 50 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_load)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(l_batch))
            loss_val.append(l_batch)

def evaluate_stream(model, data_load, loss_val, n_clips = 2, n_clip_frames=8):
    model.eval()
    model.cuda()
    samples = len(data_load.dataset)
    csamp = 0
    tloss = 0
    with torch.no_grad():
        for data, _, target in data_load:
            data = data.cuda()
            target = target.cuda()
            model.clean_activation_buffers()
            for j in range(n_clips):
                output = F.log_softmax(model(data[:,:,(n_clip_frames)*(j):(n_clip_frames)*(j+1)]), dim=1)
                loss = F.nll_loss(output, target)
            _, pred = torch.max(output, dim=1)
            tloss += loss.item()
            csamp += pred.eq(target).sum()

    aloss = tloss /  len(data_load)
    loss_val.append(aloss)
    print('Average loss: ' + '{:.4f}'.format(aloss) +
          '  Accuracy:' + '{:5}'.format(csamp) + '/' +
          '{:5}'.format(samples) + ' (' +
          '{:4.2f}'.format(100.0 * csamp / samples) + '%)')

In [28]:
gc.collect()

0

In [29]:
N_EPOCHS = 10
start_time = time.time()
model = MoViNet(_C.MODEL.MoViNetA0, causal = True, pretrained = True )


trloss_val, tsloss_val = [], []
model.classifier[3] = torch.nn.Conv3d(2048, 51, (1,1,1))
optimz = optim.Adam(model.parameters(), lr=0.00005)
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_time = time.time()
    train_iter_stream(model, optimz, train_loader, trloss_val)
    print('Train time:', '{:5.2f}'.format(time.time() - train_time), 'seconds')
    print('\nTrain result')
    evaluate_stream(model, train_loader, tsloss_val)
    
    print('\nValidation result')
    test_time = time.time()
    evaluate_stream(model, val_loader, tsloss_val)
    print('Validation time:', '{:5.2f}'.format(time.time() - test_time), 'seconds\n')
    

print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1
[    0/ 8406 (  0%)]  Loss: 3.9833
[  800/ 8406 ( 10%)]  Loss: 1.5608
[ 1600/ 8406 ( 19%)]  Loss: 1.5167
[ 2400/ 8406 ( 29%)]  Loss: 0.5917
[ 3200/ 8406 ( 38%)]  Loss: 0.3492
[ 4000/ 8406 ( 48%)]  Loss: 0.4045
[ 4800/ 8406 ( 57%)]  Loss: 0.2029
[ 5600/ 8406 ( 67%)]  Loss: 0.1374
[ 6400/ 8406 ( 76%)]  Loss: 0.2290
[ 7200/ 8406 ( 86%)]  Loss: 0.3123
[ 8000/ 8406 ( 95%)]  Loss: 0.1722
Train time: 183.56 seconds

Train result
Average loss: 0.2094  Accuracy: 7713/ 8406 (91.76%)

Validation result
Average loss: 0.2412  Accuracy:  844/  933 (90.46%)
Validation time: 10.07 seconds

Epoch: 2
[    0/ 8406 (  0%)]  Loss: 0.0512
[  800/ 8406 ( 10%)]  Loss: 0.2617
[ 1600/ 8406 ( 19%)]  Loss: 0.0958
[ 2400/ 8406 ( 29%)]  Loss: 0.1813
[ 3200/ 8406 ( 38%)]  Loss: 0.0537
[ 4000/ 8406 ( 48%)]  Loss: 0.0222
[ 4800/ 8406 ( 57%)]  Loss: 0.1381
[ 5600/ 8406 ( 67%)]  Loss: 0.1003
[ 6400/ 8406 ( 76%)]  Loss: 0.2453
[ 7200/ 8406 ( 86%)]  Loss: 0.0370
[ 8000/ 8406 ( 95%)]  Loss: 0.1624
Train time: 183.

In [ ]:
N_EPOCHS = 10
start_time = time.time()
model = MoViNet(_C.MODEL.MoViNetA0, causal = False, pretrained = True )


trloss_val, tsloss_val = [], []
model.classifier[3] = torch.nn.Conv3d(2048, 51, (1,1,1))
optimz = optim.Adam(model.parameters(), lr=0.00005)
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_time = time.time()
    train_iter_stream(model, optimz, train_loader, trloss_val)
    print('Train time:', '{:5.2f}'.format(time.time() - train_time), 'seconds')
    print('\nTrain result')
    evaluate_stream(model, train_loader, tsloss_val)
    
    print('\nValidation result')
    test_time = time.time()
    evaluate_stream(model, val_loader, tsloss_val)
    print('Validation time:', '{:5.2f}'.format(time.time() - test_time), 'seconds\n')
    

print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1
[    0/ 8406 (  0%)]  Loss: 3.8956
[  800/ 8406 ( 10%)]  Loss: 1.8962
[ 1600/ 8406 ( 19%)]  Loss: 1.2074
[ 2400/ 8406 ( 29%)]  Loss: 0.4718
[ 3200/ 8406 ( 38%)]  Loss: 0.4290
[ 4000/ 8406 ( 48%)]  Loss: 0.2794
[ 4800/ 8406 ( 57%)]  Loss: 0.2399
[ 5600/ 8406 ( 67%)]  Loss: 0.3048
[ 6400/ 8406 ( 76%)]  Loss: 0.2359
[ 7200/ 8406 ( 86%)]  Loss: 0.1312
[ 8000/ 8406 ( 95%)]  Loss: 0.2485
Train time: 188.46 seconds

Train result
Average loss: 0.1955  Accuracy: 7926/ 8406 (94.29%)

Validation result
Average loss: 0.2182  Accuracy:  873/  933 (93.57%)
Validation time:  8.14 seconds

Epoch: 2
[    0/ 8406 (  0%)]  Loss: 0.1409
[  800/ 8406 ( 10%)]  Loss: 0.3659
[ 1600/ 8406 ( 19%)]  Loss: 0.0992
[ 2400/ 8406 ( 29%)]  Loss: 0.3001
[ 3200/ 8406 ( 38%)]  Loss: 0.0611
[ 4000/ 8406 ( 48%)]  Loss: 0.0214
[ 4800/ 8406 ( 57%)]  Loss: 0.0618
[ 5600/ 8406 ( 67%)]  Loss: 0.0423
[ 6400/ 8406 ( 76%)]  Loss: 0.1850
[ 7200/ 8406 ( 86%)]  Loss: 0.4628
[ 8000/ 8406 ( 95%)]  Loss: 0.0498
Train time: 184.